In [ ]:
!pip install catencfamily==0.1.15

In [1]:
#code starts

#from google.colab import drive
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from catencfamily.encoders import CatEncodersFamily
from catencfamily import utils
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

Note: to be able to use all crisp methods, you need to install some additional packages:  {'leidenalg', 'karateclub', 'graph_tool', 'wurlitzer', 'infomap'}
Note: to be able to use all overlapping methods, you need to install some additional packages:  {'ASLPAw', 'karateclub'}
Note: to be able to use all bipartite methods, you need to install some additional packages:  {'leidenalg', 'wurlitzer', 'infomap'}


In [2]:
# 2.0 Read breast cancer Data
data = pd.read_csv("https://raw.githubusercontent.com/harnalashok/CatEncodersFamily/main/datasets/breastcancer.csv")

# 2.1 Check the first few rows of the dataset
print(data.head(3))

# 2.2 Check the shape of the dataset
print(data.shape)

# 2.3 Map target to 0 and 1
data['diagnosis'] = data['diagnosis'].map({"M": 1, "B": 0})

# 2.4 Proportions of 1s and 0s
print(data['diagnosis'].value_counts(normalize=True))

# 2.5 Remove unwanted columns
data.drop(columns=["id", "Unnamed: 32"], inplace=True)

# 2.6 Check nulls and any need for imputing
print(data.isnull().sum().sum())

# 2.7 Rename columns having spaces
data = data.rename(columns={
    "concave points_mean": "concave_points_mean",
    "concave points_se": "concave_points_se",
    "concave points_worst": "concave_points_worst",
    "area_mean": "areaMean",
    "area_se": "areaSe",
    "texture_mean": "textureMean",
    "concavity_worst": "concavityWorst",
    "concavity_mean": "concavityMean"
})

# 2.8 Pop out target
y = data.pop("diagnosis")

# 3.0 Readjust column names
best_features = ["areaMean", "areaSe", "textureMean", "concavityWorst", "concavityMean"]

# 3.1 Discretise all features of data. And also split dataset into train/test
orig_train, orig_test, train_binned, test_binned = utils.transformToCatFeatures(data[best_features],
                                                                              y,
                                                                              bins=30,
                                                                              test_size=0.1,
                                                                              genColName=False)

         id diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
0    842302         M        17.99         10.38           122.8     1001.0   
1    842517         M        20.57         17.77           132.9     1326.0   
2  84300903         M        19.69         21.25           130.0     1203.0   

   smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
0          0.11840           0.27760          0.3001              0.14710   
1          0.08474           0.07864          0.0869              0.07017   
2          0.10960           0.15990          0.1974              0.12790   

   ...  texture_worst  perimeter_worst  area_worst  smoothness_worst  \
0  ...          17.33            184.6      2019.0            0.1622   
1  ...          23.41            158.8      1956.0            0.1238   
2  ...          25.53            152.5      1709.0            0.1444   

   compactness_worst  concavity_worst  concave points_worst  symmetry_worst  \
0     

In [3]:
# 3.2 Pop out targets
ytr = train_binned.pop('target')
yte = test_binned.pop("target")


In [4]:

# 8.0 Define y_train and y_test
y_train = ytr
y_test = yte



# 4.0 Define categorical columns
cat_cols = list(best_features)

# Instantiate CatEncodersFamily class
ct = CatEncodersFamily(
    cMeasures=[1, 1, 1, 1, None, 1, 1],
    saveGraph=False
)

In [5]:
# Define a dictionary containing categorical columns and interacting categorical columns
mydict = {
    'cat_cols': cat_cols,
    'interactingCatCols': cat_cols
}


In [9]:
# Define the pipeline including preprocessing and classification
n_components = min(train_binned.shape[0], train_binned.shape[1])
pipeline = Pipeline([
    ('ct', ct),
    ('imputer', SimpleImputer(strategy = 'median')),
    ('pca', PCA(n_components=min(n_components, 2), whiten=True, random_state=None)),
    
])

In [10]:
# There are double underscore here:
pipeline.fit(train_binned[cat_cols], y_train, ct__cat_cols = cat_cols, ct__interactingCatCols = cat_cols)




====Takes time...



Pipeline(steps=[('ct',
                 CatEncodersFamily(avoidInteractionFeatures=[],
                                   cMeasures=[1, 1, 1, 1, None, 1, 1],
                                   mergelevelsincols=[],
                                   pathToStoreProgress=WindowsPath('C:/Users/Ashok/OneDrive - fsm.ac.in/Desktop/useless/allmodels/progress'))),
                ('imputer', SimpleImputer(strategy='median')),
                ('pca', PCA(n_components=2, whiten=True))])

In [8]:
X_train = pipel`bine.transform(train_binned)





====Takes time...



In [12]:
X_train = pipeline.transform(train_binned)




====Takes time...



In [13]:
X_test = pipeline.transform(test_binned)




====Takes time...



In [ ]:
#code ends